## Fake news classification (kaggle)

#### Source: https://www.kaggle.com/c/fake-news
The Project is designed to analyze and classify news sources
for authenticity and accuracy

##### Steps involved in deep learning RNN analysis

- Read the text and classification label
- Tokenize the text and clean
- Perform word embedding
- Train RNN/LSTM model



###### References:

###### - https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b
- https://github.com/msahamed/yelp_comments_classification_nlp
###### - https://towardsdatascience.com/understanding-lstm-and-its-quick-implementation-in-keras-for-sentiment-analysis-af410fd85b47
- https://github.com/nsinha280/lstm-on-Yelp-review-data/blob/master/lstm-final.ipynb

In [28]:
## Input libraries

## system libraries

import os
import sys
import glob
import re

## analysis libraries

import pandas as pd
import numpy as np


## text and language processing
from langdetect import detect
import string
#from string import punctuation
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

from sklearn.manifold import TSNE
## Keras
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


## plotting
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns
%matplotlib inline



[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [29]:
## Data source and files

dir_data = r'data'

file_train = r'data/train.csv'
file_train_clean = r'train_clean.tsv'
file_test = r'data/test.csv'

In [31]:
#### Train Data summary

train_all_df = pd.read_csv(file_train)

print(train_all_df.shape)
train_all_df.head()

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [35]:
## remove empty cells
train_all_df = train_all_df[train_all_df.text.apply(lambda x: x !="")]

print(train_all_df.shape)

(20800, 5)


In [36]:
## replace empty cells with NAN and remove rows with NA
train_all_df['text'].replace('', np.nan, inplace=True)

train_all_df.dropna(inplace=True,subset=['text','label'])

train_all_df['text'].isnull().sum()

print(train_all_df.shape)

(20761, 5)


#### Tokenize the text and clean

In [55]:

df_all = train_all_df[['text','label']]

### select rows
#df = df_all.iloc[0:500]
#labels =
df = df_all

### Create sequence
vocabulary_size = 20000

tokenizer = Tokenizer(num_words= vocabulary_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,split=' ')
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data_all = pad_sequences(sequences, maxlen=50)

In [61]:
data = data_all[:2000]
label = df.iloc[:2000,1]
len(data)
#print(data[:10])

2000

#### Build the word embedding LSTM model 

In [62]:
## Network architecture
embeding_size = 128

model = Sequential()
model.add(Embedding(20000, embeding_size, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


#### Run the model

In [68]:
## Fit the model
model.fit(data, np.array(label), validation_split=0.2, epochs=10)

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.1253 - acc: 0.9537 - val_loss: 0.3718 - val_acc: 0.8725
Epoch 2/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.0178 - acc: 0.9963 - val_loss: 0.3091 - val_acc: 0.8825
Epoch 3/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.0062 - acc: 0.9994 - val_loss: 0.3768 - val_acc: 0.8875
Epoch 4/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.0060 - acc: 0.9994 - val_loss: 0.4032 - val_acc: 0.8850
Epoch 5/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.6173 - val_acc: 0.8550
Epoch 6/10
1600/1600 [==============================] - 6s 4ms/step - loss: 5.3980e-04 - acc: 1.0000 - val_loss: 0.6028 - val_acc: 0.8600
Epoch 7/10
1600/1600 [==============================] - 6s 4ms/step - loss: 3.2056e-04 - acc: 1.0000 - val_loss: 0.6978 - val_acc: 0.8525
Epoch 8

#### Evaluate the model

In [69]:
## Evaluate the model
X_test = data_all[2000:2500]
Y_test =  df.iloc[2000:2500,1]

In [70]:

score,acc  =  model.evaluate(X_test, Y_test, verbose = 2)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.62
Validation Accuracy: 0.87


#### Testing code

In [12]:
## adapted from https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b

### Text Normalizing function. Part of the following function was taken from this link. 
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    """
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    """
    return text


In [13]:
## apply the above function to df['text']
train_all_df['text'] = train_all_df['text'].map(lambda x: clean_text(x))

In [14]:
print(train_all_df.shape)
train_all_df.head()

(20761, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide : didn t even see comey s lette...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might get fired october 29 2016 + the te...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,videos civilians killed single airstrike ident...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print + an iranian woman sentenced six years p...,1


#### Tokenize and clean documents (NLTK)

In [22]:
#example_sent = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))


def tokenize_words_nltk(input_text):
    """
    Input a text string
    - tokenize the words
    - make lower case
    - remove stop words
    - remove punctuations
    
    Otput the list of clean text string
    """
    
    ## make text lower case
    input_text = input_text.lower()

    ## tokenize text to words
    word_tokens = word_tokenize(input_text)

    ## filter for stop words
    word_filtered = [w for w in word_tokens if not w in stop_words]
    
    return(word_filtered)

In [10]:
train_all_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#### Summary statistics

In [11]:
train_all_df[['author','label','text']].groupby(['label']).count()

,author,text
label,,
0,10361,10387
1,8482,10374


In [13]:
print(train_all_df.shape)

(20761, 5)


In [14]:
train_all_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
### Function to remove punctuations

def remove_punctuation(text_str):
    """
    Takes a string input of a short article
    Outputs a string without any punctuations.
    """


In [15]:
### Detect the language of the text article in a list

def lang_detect_list(list_docs):
    """
    Check if txt exit, Check lang
    else, print n lang
    """
    lang = []
    for obj in list_docs:
        obj = obj[:100]
        if re.search('\w',obj):
        #if isinstance(obj, str) and obj is not None:
            #print('obj is a string!')
            #print(detect(obj)) # for x in train_all_df['text'][:10]]
            try:
                lang_str = detect(obj)
            except:
                lang_str = 'exception'
            
            lang.append(lang_str)

        else:
            lang.append('NoString')
            
    return(lang)

In [16]:
#train_all_df.
#lang_detect_list(train_all_df['text'][:200])

lang_list = lang_detect_list(train_all_df['text'])

In [17]:
## add the language as a column 
train_all_df['lang'] = lang_list

In [18]:
train_all_df.head()

,id,title,author,text,label,lang
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,en
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,en
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,en
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,en
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,en


In [23]:
## write the modified data as a file
out_file = dir_data + '/' + file_train_clean
train_all_df.to_csv(out_file,sep='\t')


In [4]:
#### Test Data summary

test_all_df = pd.read_csv(file_test)

print(test_all_df.shape)
test_all_df.head()

(5200, 4)


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [12]:
## Count authors with multiple fake news articles

train_all_df[['author','label']].groupby(['author']).sum()

,label
author,
# 1 NWO Hatr,17
-NO AUTHOR-,54
10 Habits That Will Make Your Life Easier &amp; More Peaceful - Wellness Solutions,1
"10 More Beautiful Images That Remind You We Still Live In A Beautiful World, With Beautiful People - Upside Down Media",1
10 Movies That Could Change Your Understanding Of Life - Upside Down Media,1
10 Nutritious Foods That Help Reduce Asthma Attacks &amp; Boost Your Health - Health Alternative Solutions,1
10 Presidents &amp; Politicians Who Told Us That A “Secret Government” Controls The World &amp; What They Said - Upside Down Media,1
10 Shocking Facts About Society That We Absurdly Accept As Normal – Digital Flow,1
11 Things To Let Go Of Before The New Year,1
